# Zindi Competition :
* Sandra Mourali
* Salma Ghabri
* Aziz Bellaaj

## Imports : 

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import csv
import gradio as gr
import os

import warnings
from tqdm import tqdm_notebook as tqdm
import itertools
import tensorflow as tf
warnings.filterwarnings("ignore")
SEED = 42  

## Model Architecture :

In [5]:
model = Sequential()
model.add(Conv2D(20,(5,5), padding="same",input_shape=(24,24,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

model.add(Dense(2))
model.add(Activation("sigmoid"))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 20)        1520      
                                                                 
 activation (Activation)     (None, 24, 24, 20)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 20)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2880)              0         
                                                                 
 dense (Dense)               (None, 500)               1440500   
                                                                 
 activation_1 (Activation)   (None, 500)               0         
                                                        

In [42]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth) 

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        model.add(Conv2D(20, (5, 5), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Conv2D(100, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))

        model.add(Dense(classes))
        model.add(Activation("sigmoid"))

        return model


## Train-Test Split and Data-Augmentation

In [53]:
DATASET = "train"  
MODEL = "zindi.model" 
PLOT = "plot.png" 

EPOCHS = 50
INIT_LR = 1e-3
BS = 32

data = []
labels = []

In [10]:
with open('train.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader) 
    image_label_map = {row[0]: int(float(row[1])) for row in csv_reader}

imagePaths = sorted(list(paths.list_images(DATASET)))
random.seed(42)
random.shuffle(imagePaths)

with tqdm(total=len(imagePaths)) as pbar:
    for idx, imagePath in enumerate(imagePaths):
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (28, 28))
        image = img_to_array(image)
        data.append(image)

        image_file = imagePath.split(os.path.sep)[-1]
        label = image_label_map.get(image_file, -1)  

        labels.append(label)
        pbar.update(1)


[INFO] loading images...


  0%|          | 0/2603 [00:00<?, ?it/s]

In [13]:
# data[0]
# data[0].shape
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
# labels

In [18]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)
# trainX.shape
# trainY.shape
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)
trainY[0]

In [20]:
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2,
                         horizontal_flip=True,
                         fill_mode="nearest")

## Model Training :

In [54]:
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
# print(model.summary())

In [23]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [24]:
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
              validation_data=(testX, testY),
              steps_per_epoch=len(trainX) // BS,
              epochs=200,
              verbose=1)

[INFO] training network...
Epoch 1/200
61/61 [==============================] - 6s 96ms/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/200
61/61 [==============================] - 5s 78ms/step - loss: 3.2729e-36 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/200
61/61 [==============================] - 5s 84ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/200
61/61 [==============================] - 5s 83ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/200
61/61 [==============================] - 5s 79ms/step - loss: 4.7650e-34 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/200
61/61 [==============================] - 5s 76ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/200
61/61 [==============================] - 5s 80ms/step - loss: 0.0

In [55]:
model.save(MODEL)

INFO:tensorflow:Assets written to: zindi.model\assets


INFO:tensorflow:Assets written to: zindi.model\assets


## Model Evaluation :

In [77]:
# N = np.arange(0, 200)
# plt.style.use("ggplot")
# plt.figure(figsize = [10,8])
# plt.plot(N, H.history["accuracy"], label="train_acc")
# plt.plot(N, H.history["val_accuracy"], label="val_acc")
# plt.title("CNN: Training and Validation Accuracy")
# plt.xlabel("Epoch #", weight="bold")
# plt.ylabel("Accuracy", weight="bold")
# plt.legend()
# plt.show()

In [78]:
# N = np.arange(0, 200)
# plt.style.use("ggplot")
# plt.figure(figsize = [10,8])
# plt.plot(N, H.history["loss"], label="train_loss")
# plt.plot(N, H.history["val_loss"], label="val_loss")
# plt.title("CNN: Training & Validation Loss")
# plt.xlabel("Epoch #", weight="bold")
# plt.ylabel("Loss", weight="bold")
# plt.legend()
# plt.show()

## Model Testing :

#### Submission.csv :

In [79]:
from tqdm import tqdm
from tensorflow.keras.models import load_model
import pickle
import cv2
import numpy as np
import os

model = load_model(MODEL)
testImagePaths = sorted(list(paths.list_images('test')))
all_class = ["1", "0"]

def predict_image(image):
    image = cv2.resize(image, (28, 28))
    image = image.astype("float") / 255.0
    image = np.expand_dims(image, axis=0)

    preds = model.predict(image)[0]
    result = dict()
    result["1"] = round(float(preds[0]), 2)
    result["0"] = round(float(preds[1]), 2)

    return result

submission_file = 'submission.csv'

with open(submission_file, 'w') as file:
    file.write('ImageId,Target\n')
    for image_path in tqdm(testImagePaths):
        image_name = os.path.basename(image_path)
        image = cv2.imread(image_path)
        result = predict_image(image)
        if result["1"] > 0.5:
            file.write(f'{image_name},1\n')
        else:
            file.write(f'{image_name},0\n')

print(f'Submission file "{submission_file}" created successfully.')


  0%|          | 0/196 [00:00<?, ?it/s]

1/1 [==============================] - 0s 17ms/step


  2%|▏         | 3/196 [00:00<00:08, 22.92it/s]

1/1 [==============================] - 0s 16ms/step


  3%|▎         | 6/196 [00:00<00:08, 23.65it/s]

...

1/1 [==============================] - 0s 26ms/step


 99%|█████████▉| 195/196 [00:08<00:00, 19.72it/s]

1/1 [==============================] - 0s 15ms/step


100%|██████████| 196/196 [00:08<00:00, 21.80it/s]

Submission file "submission.csv" created successfully.
